In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import f

# Load data from Excel files

# Load data from Excel files
FF2530Z_1992 = pd.read_excel(r"C:\Users\USER\OneDrive\Escritorio\Memoria\Memoria_LA\Fama-macBeth data\Portfolio_returns_10_EW.xlsx", header=None)
CAPMfactors_1992 = pd.read_excel(r"C:\Users\USER\OneDrive\Escritorio\Memoria\Memoria_LA\Fama-macBeth data\Reg_factors.xlsx", header=None)

# Convert to numeric values
FF2530Z_1992 = FF2530Z_1992.values
CAPMfactors_1992 = CAPMfactors_1992.values

# Calculate excess returns
Rf = np.zeros((len(CAPMfactors_1992), 1))  # DataFrame filled with zeros
ff = FF2530Z_1992 - Rf

# First Pass
facFF = np.column_stack((np.ones(len(CAPMfactors_1992)), CAPMfactors_1992))
betaFF = np.zeros((55, 3))
tvalueFF = np.zeros((55, 3))
adjR2FF = np.zeros(55)
errFF = np.zeros((len(CAPMfactors_1992), 55))

for i in range(55):
    result3pass = np.linalg.lstsq(facFF, ff[:, i], rcond=None)
    betaFF[i, :] = result3pass[0]
    predictions = np.dot(facFF, betaFF[i, :])
    residuals = ff[:, i] - predictions
    errFF[:, i] = residuals
    adjR2FF[i] = 1 - residuals.var() / ff[:, i].var()

RbarpFF = adjR2FF.mean()
L = 1
alpha = betaFF[:, 0]
sigma = np.cov(errFF) * (len(CAPMfactors_1992) - 1) / (len(CAPMfactors_1992) - L - 1)
omega = np.cov(facFF[:, 1:].T)
sigmaFull = np.cov(ff.T)
Rp_bar = facFF[:, 1:].mean(axis=0)

GRS_F = (len(CAPMfactors_1992) / 55) * ((len(CAPMfactors_1992) - 55 - L) / (len(CAPMfactors_1992) - L - 1)) * \
        ((alpha @ np.linalg.inv(sigma) @ alpha) / (1 + Rp_bar @ np.linalg.inv(omega) @ Rp_bar))
GRS_F2 = (len(CAPMfactors_1992) / 55) * ((len(CAPMfactors_1992) - 55 - L) / (len(CAPMfactors_1992) - L - 1)) * \
         ((alpha @ np.linalg.inv(sigmaFull) @ alpha) / (1 + Rp_bar @ np.linalg.inv(omega) @ Rp_bar))

print('Significance level of the F test is:', f.cdf(GRS_F, 55, len(CAPMfactors_1992) - 55 - L))

# The critical values
print('The critical values of the F tests of 95% and 99% are:')
print(f.ppf(0.95, 55, len(CAPMfactors_1992) - 55 - L), f.ppf(0.99, 55, len(CAPMfactors_1992) - 55 - L))

alphaabs = np.abs(alpha)
avgerror = np.mean(alphaabs)
RbarpFF = np.mean(adjR2FF)
GRS_F = (len(CAPMfactors_1992) / 55) * ((len(CAPMfactors_1992) - 55 - L) / (len(CAPMfactors_1992) - L - 1)) * \
        ((alpha @ np.linalg.inv(sigma) @ alpha) / (1 + Rp_bar @ np.linalg.inv(omega) @ Rp_bar))
p_value_F = 1 - f.cdf(GRS_F, 55, len(CAPMfactors_1992) - 55 - L)

# Second Pass (for average returns)
betaFF[:, 0] = 1
result2pass = np.linalg.lstsq(betaFF, ff.mean(axis=0), rcond=None)
RP2p = result2pass[0]
RPtvalue2p = [0] * 3  # Placeholder for t-values
adjR22p = 1 - result2pass[1] / ff.mean(axis=0).var()
pred_return = np.dot(betaFF, RP2p)

print('     mean      tstat')
print(RP2p, RPtvalue2p)
print('Adjusted R squared=', adjR22p)

xxx = [0, 2]
plt.scatter(pred_return, ff.mean(axis=0), color='r', alpha=0.5)
plt.title('CAPM - FF25+FF30 industries (1927-2014 monthly excess returns)')
plt.xlabel('predicted excess return')
plt.ylabel('actual excess return')
plt.plot(xxx, xxx, color='blue')
plt.show()

# Second Pass (cross-sectional reg. for every month)
betaFF[:, 0] = 1
RP2p = np.zeros((len(ff), 3))
RPtvalue2p = np.zeros((len(ff), 3))
R22p = np.zeros(len(ff))
adjR22p = np.zeros(len(ff))
err2p = np.zeros((len(ff), 55))
pred_return = np.zeros(len(ff))

for i in range(len(ff)):
    result2mpass = np.linalg.lstsq(betaFF, ff[i, :], rcond=None)
    RP2p[i, :] = result2mpass[0]
    R22p[i] = 1 - result2mpass[1] / ff[i, :].var()
    adjR22p[i] = 1 - (1 - R22p[i]) * (T - 1) / (T - 2)
    predictions = np.dot(betaFF, RP2p[i, :])
    residuals = ff[i, :] - predictions
    err2p[i, :] = residuals
    pred_return[i] = predictions.mean()

meanRP = RP2p.mean(axis=0)
tstat = np.sqrt(T) * meanRP / RP2p.std(axis=0)
Rbarp22 = adjR22p.mean()
stdRbar = adjR22p.std()
report = np.column_stack((meanRP, tstat))
print('     mean      tstat')
print(report)
report2 = np.column_stack((Rbarp22, stdRbar))
print('Avg.Adj.R2  Std.dev. Adj.R2')
print(report2)

xxx = [0, 2]
plt.scatter(pred_return, ff.mean(axis=1), color='r', alpha=0.5)
plt.title('CAPM - FF25+FF30 industries (1927-2014 monthly excess returns)')
plt.xlabel('predicted excess return')
plt.ylabel('actual excess return')
plt.plot(xxx, xxx, color='blue')
plt.show()


0     object
1     object
2     object
3     object
4     object
5     object
6     object
7     object
8     object
9     object
10    object
11    object
12    object
13    object
14    object
15    object
16    object
17    object
18    object
19    object
20    object
21    object
22    object
23    object
dtype: object


TypeError: unsupported operand type(s) for -: 'str' and 'float'